In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Reading Datasets using pandas read_csv
train = pd.read_csv('../input/jobathon-june-2022/train_wn75k28.csv')
test = pd.read_csv('../input/jobathon-june-2022/test_Wf7sxXF.csv')
sample = pd.read_csv('../input/jobathon-june-2022/sample_submission_2zvVjBu.csv')

In [3]:
#output of train dataframe
train

In [4]:
#output of test dataframe
test

In [5]:
#Knowing number of null values in train dataset
train.isnull().sum()

In [6]:
#Knowing number of null values in test dataset
test.isnull().sum()

In [7]:
# output of train dataset information
train.info()

In [8]:
#Output of rows with null values in signup_date column
train[train['signup_date'].isna()]

In [9]:
# Converting prod_flag column to int type wherever values are null
# train['prod_flag'] = train['products_purchased'].isna().astype('int')
# test['prod_flag'] = test['products_purchased'].isna().astype('int')

In [10]:
# Converting signupdate feature to same as created_at wherever its null
#Converting Products purchased to zero wherever its null
train['signup_date'].fillna(train['created_at'], inplace = True)
test['signup_date'].fillna(test['created_at'], inplace = True)
train['products_purchased'].fillna(0, inplace = True)
test['products_purchased'].fillna(0, inplace = True)
train['products_purchased'] = train['products_purchased'].astype('int')
test['products_purchased'] = test['products_purchased'].astype('int')

In [11]:
# Output of sum of null values in signup_date column 
train['signup_date'].isna().sum()

In [12]:
# stats of signup_date column in train dataset
train['signup_date'].describe()

In [13]:
train['signup_date']

In [14]:
train

In [15]:
train['campaign_var_2'].describe()

In [16]:
#One-hot encoding
train = pd.get_dummies(train, columns=['campaign_var_1', 'campaign_var_2'])
test = pd.get_dummies(test, columns=['campaign_var_1', 'campaign_var_2'])

In [17]:
#Creating new feature difference of date features in days
train[['created_at','signup_date']] = train[['created_at','signup_date']].apply(pd.to_datetime) #if conversion required
train['diff_date'] = np.abs((train['created_at'] - train['signup_date']).dt.days)

In [18]:
# train['diff_date']

In [19]:
test[['created_at','signup_date']] = test[['created_at','signup_date']].apply(pd.to_datetime) #if conversion required
test['diff_date'] = np.abs((test['created_at'] - test['signup_date']).dt.days)

In [20]:
# Dropping date features
train.drop(['id','created_at','signup_date'], axis = 1, inplace = True)

In [21]:
test.drop(['id','created_at','signup_date'], axis = 1, inplace = True)

In [22]:
train

In [23]:
X = train.drop('buy', axis = 1)
y = train['buy']

In [24]:
y.value_counts()

In [25]:
X.columns

In [26]:
test.columns

In [27]:
# !pip install imblearn

In [28]:
# from imblearn.ensemble import BalancedRandomForestClassifier
# brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
# brf.fit(X, y)
# y_pred = brf.predict(test)

In [29]:
# Selecting K Best features using SelectKBest
from sklearn.feature_selection import SelectKBest, chi2
skb = SelectKBest(chi2, k=10)
X_new = skb.fit_transform(X, y)
test_new = skb.transform(test)

In [30]:
from xgboost import XGBClassifier
from collections import Counter
counter = Counter(y)
w = counter[0]/counter[1]
# scale_pos_weight=w
model = XGBClassifier()
model.fit(X_new, y)
y_pred = model.predict(test_new)

In [31]:
sample['buy'] = y_pred

In [32]:
sample['buy'].value_counts()

In [33]:
test.shape

In [34]:
test_new.shape

In [35]:
sample.to_csv('nine_xgb_submn.csv', index = False)